WINDOWS UPDATES AUSSCHALTEN!

In [1]:
# --- Python & Data Libraries ---
print("Importing Libraries")
import pandas as pd
import sys
import os
import time
import numpy as np
import shutil
from collections import deque
import ctypes
from datetime import datetime


# --- Path Setup ---
# Add the 'hardware_drivers' folder to the path so Python can find your new scripts
# Get the current folder path (experiments/)
current_dir = os.getcwd()
# Get the parent folder path (Project_Root/)
parent_dir = os.path.dirname(current_dir)
driver_path = os.path.join(parent_dir, 'hardware_drivers')
if driver_path not in sys.path:
    sys.path.append(driver_path)

Importing Libraries


In [2]:
# --- Spectrometer Setup (OceanDirect) ---
print("\nSetting up Spectrometers (OceanDirect)")
from oceandirect.OceanDirectAPI import OceanDirectAPI, OceanDirectError, Spectrometer
# Initialize
od = OceanDirectAPI()

# Device Discovery Logic 
device_count = od.find_usb_devices()
if device_count > 0:
    device_ids = od.get_device_ids()
    deviceid_infos_dict = {}
    for deviceid in device_ids:
        deviceid_infos_dict[deviceid] = None
        device_infos_list = []
        device = od.open_device(deviceid)
        device_type = device.get_device_type()
        device_infos_list.append(device_type)
        device_serial_number = device.get_serial_number()        
        device_infos_list.append(device_serial_number)
        deviceid_infos_dict[deviceid] = device_infos_list
print(deviceid_infos_dict)

# Assign NIR and VIS spectrometers
for key, value in deviceid_infos_dict.items():
    if value[1] == "NQ51B1182":
        nir_deviceid = key
        nir = od.open_device(nir_deviceid)
    if value[1] == "QEA0031":
        vis_deviceid = key
        vis = od.open_device(vis_deviceid)

# Ensure NIR/VIS objects exist
if 'nir' not in locals() or 'vis' not in locals():
    raise RuntimeError("Spectrometer initialization failed. Check device IDs.")

# Activate cooling for NIR spectrometer
print("Activating cooling for NIR spectrometer")
nir.Advanced.set_tec_enable(True)
nir.Advanced.set_temperature_setpoint_degrees_C(-25)

current_tec_temperature = 25
while current_tec_temperature > -20:
    current_tec_temperature = nir.Advanced.get_tec_temperature_degrees_C()
    print("TEC temperature: %f" % current_tec_temperature)
    if current_tec_temperature < -20:
        break
    time.sleep(5)

nir.set_electric_dark_correction_usage(False)
nir.set_nonlinearity_correction_usage(False)
vis.set_electric_dark_correction_usage(False)
vis.set_nonlinearity_correction_usage(False)


Setting up Spectrometers (OceanDirect)
{2: ['NIRQUEST512', 'NQ51B1182'], 3: ['QE65000', 'QEA0031']}
Activating cooling for NIR spectrometer
TEC temperature: -23.400000


In [3]:
# --- Import Your New Drivers ---
from PressureController_OB import PressureController
from ValveController_MuxWire import MuxWire
from MultiValve_MuxDistribution import MuxDistribution
from StirrerController import StirrerController
from FlowControl import FlowControl, FlowControlError

# --- Device Configuration Constants ---
OB1_SN = '02079C06' 
MUX_COM = 'COM11'
MUXD_COM = 'COM5'
STIRRER_COM = 'COM8'


# --- Instantiation ---
try:
    global ob, mux, muxd, stirrer, flow 

    # Instantiate low-level controllers
    ob = PressureController(device_name_or_serial=OB1_SN)
    mux = MuxWire(device_name=MUX_COM)
    muxd = MuxDistribution(com_port=MUXD_COM)

    if 'stirrer' in globals():
        try:
            stirrer.close(); print("   -> Closed previous Stirrer.")
            time.sleep(1)
        except: pass

    stirrer = StirrerController(port=STIRRER_COM)
    
    # Instantiate Protocol Manager (Using FlowControlProtocol class)
    flow = FlowControl( 
        ob=ob, 
        mux=mux, 
        muxd=muxd,
        # Placeholder constants (will be updated after calibration)
        s_slope=0.0, s_intercept=0.0,
        ag_slope=0.0, ag_intercept=0.0,
        tol_slope=0.0, tol_intercept=0.0
    )
    
except Exception as e:
    print(f"❌ Failed to initialize one or more devices: {e}")
    raise 

# --- Set Initial Safe States ---
print("\nSetting initial safe states...")
muxd.home()
muxd.switch_valve(11)

mux.set_trigger_out(False)
mux.close_all()

ob.set_trigger_out(False)
ob.set_pressure(1, 0) 
ob.set_pressure(2, 0) 

# Stirrer Status Check
stirrer.cmd_hello()
stirrer_on, temp_on = stirrer.cmd_info()
speed_set, real_speed, temp_set, real_temp = stirrer.cmd_sta()
print(f"✅ Setup Complete.")
print(f"   Stirring Status: {'ON' if stirrer_on else 'OFF'} (Set: {speed_set} rpm)")
print(f"   Heating Status: {'ON' if temp_on else 'OFF'} (Set: {temp_set} °C)")

Initializing OB1 (02079C06)...
✅ OB1 initialized. ID: 0
Initializing MUX Wire on COM11...
✅ MUX Wire initialized. ID: 0
Initializing MUX Distributor on COM5...
✅ MUX Distributor initialized. ID: 0
✅ Stirrer connected on COM8

Setting initial safe states...
🔄 Homing valve...
Home command sent. Waiting for completion...
✅ Homing complete. Reached position 1.
Switching to valve 11 (short)...
✅ Reached valve 11
✅ Setup Complete.
   Stirring Status: ON (Set: 100 rpm)
   Heating Status: ON (Set: 25.0 °C)


Next comes the calibration of the pressure controller.
Make sure that vacuum pump and air pressure are properly running and both pressure channels are closed.

In [4]:
# --- Folder Creation Helper Function ---
def create_folder(directory, number, ratio, dilution, frequency):
    timestamp = pd.Timestamp.now()
    timestamp_save = timestamp.strftime("%Y-%m-%d_%H-%M-%S")
    
    # Format numbers to be filesystem-safe (replace dots with dashes)
    ratio_str = str(ratio).replace('.', '-')
    frequency_str = str(frequency).replace('.', '-')
    dilution_str = str(dilution).replace('.', '-')
    
    # New Format: 01_10-0_Dil-1_f1_2023...
    folder_name = f"{number}_{ratio_str}_Dil-{dilution_str}_f{frequency_str}_{timestamp_save}"
    
    folder_path = os.path.join(directory, folder_name)
    
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        
    return folder_path

In [5]:
# --- CONFIGURATION ---
calibration_type = "new"  # Change to "load" to skip calibration
#calibration_type = "load"

file_name_load = "Calib_2025-11-28_19-41-56" # Use exact name

# Setup Paths
calib_folder = r"C:\kinetic_setup\git\kinetic-setup\calib"
os.makedirs(calib_folder, exist_ok=True) # Creates folder if missing

# Setup Filename
if calibration_type == 'new':
    timestamp_calib = pd.Timestamp.now()
    file_name = f"Calib_{timestamp_calib.strftime('%Y-%m-%d_%H-%M-%S')}"
    print(f"Creating New Calibration: {file_name}")
elif calibration_type == 'load':
    file_name = file_name_load
    print(f"Loading Existing: {file_name}")

calib_path = os.path.join(calib_folder, file_name)

# --- EXECUTE ---
if calibration_type == 'load':
    # Load Mode
    ob.calibrate(calib_path, load_existing=True)
else:
    # New Mode (Runs: Calib -> Save -> Verify Load)
    # Note: Ensure OB1 inlets are BLOCKED with caps!
    ob.calibrate(calib_path, load_existing=False)

print("OB1 Calibration Sequence Complete")

Creating New Calibration: Calib_2025-12-01_14-37-17
Starting NEW calibration (Ensure ALL channels have caps!)...
Physical calibration successful. Saving to file...
Performing self-test (Reloading file)...
✅ VERIFICATION PASSED: File saved and re-loaded successfully.
OB1 Calibration Sequence Complete


calibrate volumes

In [8]:
# Check Precursor Reservoirs
flow.switch_v1_tol_or_pre("pre")
ob.set_pressure(1, 1000)
time.sleep(10)
ob.set_pressure(1, 0)
time.sleep(15)
print("Precursor reservoir pressure cycle complete.")

Precursor reservoir pressure cycle complete.


In [9]:
# Check Toluene Reservoir
flow.switch_v1_tol_or_pre("tol")
ob.set_pressure(1, 1000)
time.sleep(10)
ob.set_pressure(1, 0)
time.sleep(15)
print("Toluene reservoir pressure cycle complete.")

Toluene reservoir pressure cycle complete.


In [20]:
flow.switch_v1_tol_or_pre("pre")
ob.set_pressure(1, 1000)
time.sleep(10)

In [10]:
flow.switch_v1_tol_or_pre("tol")
ob.set_pressure(1, 1000)
time.sleep(10)

In [ ]:
# Vent Pressure
ob.set_pressure(1, 0)
time.sleep(5)

In [19]:
# vary this
flow.toluene_cuvette(0)

In [29]:
# vary this
flow.S_cuvette(0)

In [38]:
# vary this
flow.Ag_cuvette(0)

In [46]:
def format_calibration_data(raw_text):
    
    # Split by lines and iterate
    lines = raw_text.strip().split('\n')
    
    for i, line in enumerate(lines):
        # Split line by whitespace (tabs or spaces)
        parts = line.split()
        
        # Skip empty lines or lines with less than 2 columns
        if len(parts) < 2:
            continue
            
        try:
            # Attempt to parse numbers
            # Using float() handles both '4' and '0.5'
            key = float(parts[0])
            val = float(parts[1])
            
            # Format the key to look like an integer if it has no decimal
            key_str = f"{int(key)}" if key.is_integer() else f"{key}"
            
            # Add comma if it's not the last item
            comma = "," if i < len(lines) - 1 else ""
            
            # Print formatted line
            print(f"    {key_str:<5}: [{val}],")
            
        except ValueError:
            # This catches headers like "time 1" and skips them
            continue

# --- Usage Example ---
raw_input ="""
time	1
4	2085.3
2	1077.2
1	574.1
0.5	319.4
0.2	168.6
0.1	119.2
0.05	93.1
0	69.3
"""

format_calibration_data(raw_input)

    4    : [2085.3],
    2    : [1077.2],
    1    : [574.1],
    0.5  : [319.4],
    0.2  : [168.6],
    0.1  : [119.2],
    0.05 : [93.1],
    0    : [69.3],


In [47]:
#Calibration
# Standard constants (UPPER_CASE for constants)
DENSITY_AG = 0.849
DENSITY_S = 0.869
DENSITY_TOL = 0.869

# Raw mass readings
calib_data_S = {
    4    : [1940.5],
    2    : [1001.0],
    1    : [531.9],
    0.5  : [298.7],
    0.2  : [156.5],
    0.1  : [111.4],
    0.05 : [86.4],
    0    : [62.3],
}
calib_data_Ag = {
    5    : [1973.6],
    2.5  : [1011.4],
    1    : [426.1],
    0.5  : [245.0],
    0.2  : [129.5],
    0.1  : [90.5],
    0.05 : [71.6],
    0    : [51.5],
}
calib_data_Tol = {
    4    : [2085.3],
    2    : [1077.2],
    1    : [574.1],
    0.5  : [319.4],
    0.2  : [168.6],
    0.1  : [119.2],
    0.05 : [93.1],
    0    : [69.3],
}

# --- RUN CALIBRATION ---
print("Running Calibration Calculations:")

# Calculate and store the results in variables
# Note: We return (slope, intercept)
slope_ag, intercept_ag = flow.run_calibration(calib_data_Ag, DENSITY_AG, "Ag")
slope_s, intercept_s = flow.run_calibration(calib_data_S, DENSITY_S, "S")
slope_tol, intercept_tol = flow.run_calibration(calib_data_Tol, DENSITY_TOL, "Toluene")

#save the data in the folder

print("\nApplying calculated volume calibration constants...")

# Send these values to the flow controller
flow.set_calibration('s', slope_s, intercept_s)
flow.set_calibration('ag', slope_ag, intercept_ag)
flow.set_calibration('tol', slope_tol, intercept_tol)

Running Calibration Calculations:
🔧 Ag Calibration Results:
   V = 452.606 * t + 59.827 (R²: 1.0000)
🔧 S Calibration Results:
   V = 540.005 * t + 72.637 (R²: 1.0000)
🔧 Toluene Calibration Results:
   V = 580.294 * t + 78.774 (R²: 1.0000)

Applying calculated volume calibration constants...
✅ S calibration updated: 540.0050x + 72.6374
✅ Ag calibration updated: 452.6065x + 59.8266
✅ Tol calibration updated: 580.2938x + 78.7743


In [ ]:
# Waste the cuvette
flow.cuvette_waste_or_tube(12, 15)

In [54]:
#IMPORTANT
# Define duration for cleaning steps
toluene_cleaning = 4.25

In [55]:
# Check cleaning procedure 

flow.toluene_cuvette(toluene_cleaning)
time.sleep(5)
flow.cuvette_waste_or_tube(12, 15)

In [53]:
flow.Ag_cuvette_volume(100)
flow.S_cuvette_volume(100)

In [56]:
# Cleaning 3 times
flow.cuvette_waste_or_tube(12, 15)
for i in range(3):
    flow.toluene_cuvette(toluene_cleaning)
    flow.cuvette_waste_or_tube(12, 15)

In [57]:
# Final cleanup
ob.set_pressure(1, 0)

In [61]:
# --- 1. CONFIGURATION ---
total_volume = 2000
Ag_concentration = 66
S_concentration = 66

calibrated_min_time_Ag, calibrated_max_time_Ag = 0.1, 5.0
calibrated_min_time_S, calibrated_max_time_S = 0.1, 4.0
calibrated_min_time_Tol, calibrated_max_time_Tol = 0.1, 4.0

# --- 2. GENERATE RATIOS ---
ratio_range1 = np.arange(10, 1, -1)     
ratio_range2 = np.arange(2, 0.9, -0.25)   
ratio_range3 = np.arange(1, 0.05, -0.1)   

ratios = np.concatenate([ratio_range1, ratio_range2, ratio_range3])

# Create the base DataFrame of unique ratios
df_base = pd.DataFrame({'ratio': ratios})
df_base['ratio'] = df_base['ratio'].round(2)
df_base = df_base.drop_duplicates(subset='ratio', keep='first').reset_index(drop=True)

# --- 3. GENERATE DILUTION BATCHES ---
dilution_factors = [1] 
batch_list = []

for factor in dilution_factors:
    temp_df = df_base.copy()
    temp_df['dilution_factor'] = factor
    batch_list.append(temp_df)

# Combine batches
df = pd.concat(batch_list, ignore_index=True)

# SORTING LOGIC: Interleaved
# This runs Ratio 10(Dil 1) -> Ratio 10(Dil 2) -> Ratio 9(Dil 1)...
df = df.sort_values(by=['ratio', 'dilution_factor'], ascending=[False, True]).reset_index(drop=True)

# --- 4. CALCULATE VOLUMES ---
# Active volume is the total volume divided by dilution
active_volume = total_volume / df['dilution_factor']
df['toluene_volume'] = total_volume - active_volume 

df['Ag_volume'] = (active_volume * df['ratio'] * S_concentration) / \
                  (Ag_concentration + df['ratio'] * S_concentration)

df['S_volume'] = (Ag_concentration * df['Ag_volume']) / \
                 (df['ratio'] * S_concentration)

# Rounding
df['Ag_volume'] = df['Ag_volume'].round(2)
df['S_volume'] = df['S_volume'].round(2)
df['toluene_volume'] = df['toluene_volume'].round(2)


# --- 5. CALIBRATION CHECKS ---
def volume_to_time(volume, slope, intercept):
    # Handle cases where volume is 0 or NaN
    if slope == 0: return 0
    return np.where(volume > 0, (volume - intercept) / slope, 0)

# Calculate durations using the variables defined in the Calibration Cell
# CRITICAL FIX: Changed CALIB_AG_SLOPE -> slope_ag, etc.
df['dur_Ag'] = volume_to_time(df['Ag_volume'], slope_ag, intercept_ag)
df['dur_S'] = volume_to_time(df['S_volume'], slope_s, intercept_s)
df['dur_Tol'] = volume_to_time(df['toluene_volume'], slope_tol, intercept_tol)

# Create Masks
mask_Ag = (df['dur_Ag'] >= calibrated_min_time_Ag) & (df['dur_Ag'] <= calibrated_max_time_Ag)
mask_S = (df['dur_S'] >= calibrated_min_time_S) & (df['dur_S'] <= calibrated_max_time_S)

# Allow Toluene to be valid if time is good OR if volume is 0
mask_Tol = ((df['dur_Tol'] >= calibrated_min_time_Tol) & (df['dur_Tol'] <= calibrated_max_time_Tol)) | \
           (df['toluene_volume'] == 0)

# Apply Filters
df_clean = df[mask_Ag & mask_S & mask_Tol].copy()

# REPORT DROPPED ROWS
dropped_rows = df[~(mask_Ag & mask_S & mask_Tol)]
if not dropped_rows.empty:
    print(f"⚠️ WARNING: {len(dropped_rows)} reactions were dropped due to calibration limits.")
else:
    print("✅ All reactions passed calibration checks.")

# Reset index and regenerate IDs
df = df_clean.reset_index(drop=True)

# --- 6. ADD FINAL CONSTANTS ---
df['frequency'] = 1
df['duration'] = 7200
df['tube_number'] = 12
df.loc[df['ratio'] <= 1 , 'duration'] = 600

# --- 7. FINAL OUTPUT & STATS ---
print("\n--- Volume Summary ---")
print(f"Total Ag Volume: {df['Ag_volume'].sum():.2f}")
print(f"Total S Volume:  {df['S_volume'].sum():.2f}")

total_seconds = df['duration'].sum()
buffer_seconds = len(df) * (3 * 60)
completion_time = pd.Timestamp.now() + pd.Timedelta(seconds=total_seconds + buffer_seconds)

print(f"Estimated Completion: {completion_time.strftime('%Y-%m-%d %H:%M:%S')}")

# Create ID and Dictionary
df['reaction_id'] = df.index + 1
reaction_parameters = df.set_index('reaction_id').to_dict('index')

# --- VERIFY ORDER ---
print("\n--- Execution Order Check (First 10) ---")
print(df[['reaction_id', 'ratio', 'dilution_factor', 'Ag_volume']])

✅ All reactions passed calibration checks.

--- Volume Summary ---
Total Ag Volume: 25168.67
Total S Volume:  18831.33
Estimated Completion: 2025-12-02 18:20:44

--- Execution Order Check (First 10) ---
    reaction_id  ratio  dilution_factor  Ag_volume
0             1  10.00                1    1818.18
1             2   9.00                1    1800.00
2             3   8.00                1    1777.78
3             4   7.00                1    1750.00
4             5   6.00                1    1714.29
5             6   5.00                1    1666.67
6             7   4.00                1    1600.00
7             8   3.00                1    1500.00
8             9   2.00                1    1333.33
9            10   1.75                1    1272.73
10           11   1.50                1    1200.00
11           12   1.25                1    1111.11
12           13   1.00                1    1000.00
13           14   0.90                1     947.37
14           15   0.80          

In [58]:
# Prevent Windows from sleeping or restarting
ctypes.windll.kernel32.SetThreadExecutionState(0x80000002)

-2147483648

In [ ]:
# --- CONFIGURATION ---

minimum_stop_time = 300  # Minimum time (in seconds) before stop condition can trigger
nir_integration_time_seconds = 0.2
vis_integration_time_seconds = 0.2
stirring_speed = 400

# Stop Condition
intensity_threshold_percentage = 0.01
time_threshold = 60
last_mean_nir = deque(maxlen=time_threshold)

# Spectrometer Setup
nir.set_integration_time(int(nir_integration_time_seconds*1e6))
vis.set_integration_time(int(vis_integration_time_seconds*1e6))

# --- START OF MAIN LOOP ---
print("🚀 Starting Main Experiment Loop...")

try:
    stirrer.start_stirring(int(stirring_speed)) 

    # --- Data and Folder Setup ---
    base_directory = r"C:\data\arelling"  
    timestamp = pd.Timestamp.now()
    date_str = timestamp.strftime("%Y-%m-%d")
    timestamp_save = timestamp.strftime("%Y-%m-%d_%H-%M-%S")
    
    daily_directory = os.path.join(base_directory, date_str)
    if not os.path.exists(daily_directory):
        os.makedirs(daily_directory)

    # Initialize 'num_measurements' in the dictionary for tracking
    for reaction_id, reaction in reaction_parameters.items():
        reaction['num_measurements'] = 0

    # Save initial parameters to CSV
    df_reaction_parameters = pd.DataFrame.from_dict(reaction_parameters, orient="index")
    df_reaction_parameters.index.name = "reaction_id"
    path_reaction_parameters = f'reaction_parameters_{timestamp_save}.csv'
    df_reaction_parameters.to_csv(os.path.join(daily_directory, path_reaction_parameters))

    # This creates a summary file of the calibration
    calibration_summary = {
        'Ag':      {'Slope': slope_ag,  'Intercept': intercept_ag},
        'S':       {'Slope': slope_s,   'Intercept': intercept_s},
        'Toluene': {'Slope': slope_tol, 'Intercept': intercept_tol},
        # You can add other global settings here too if you want:
        'General':      {'Stirring_Speed': stirring_speed, 'Stop_Threshold': intensity_threshold_percentage}
    }

    df_calibration = pd.DataFrame.from_dict(calibration_summary, orient='index')
    df_calibration.index.name = "Parameter_Type"
    
    # Save to CSV
    path_calibration = f'pre_calibration_{timestamp_save}.csv'
    df_calibration.to_csv(os.path.join(daily_directory, path_calibration))
    
    print(f"✅ Setup Complete. Parameters and Calibration saved to {daily_directory}")
    # Iterate directly over the keys (Pythonic way)
    reaction_ids = list(reaction_parameters.keys())
    
    # Using an iterator allows us to retry the CURRENT reaction without complex index math
    for reaction_id in reaction_ids:
        try:
            # --- Parameter Extraction ---
            params = reaction_parameters[reaction_id]
            ratio = params['ratio']
            dilution = params['dilution_factor'] # New parameter
            frequency = params['frequency']
            duration = params['duration']
            tube_number = params['tube_number']
            Ag_volume = params['Ag_volume']
            S_volume = params['S_volume']
            toluene_volume = params.get('toluene_volume', 0) # Use .get() for safety

            # Calculate Sleep Time
            sleep_time = frequency - nir_integration_time_seconds - vis_integration_time_seconds
            sleep_time = max(0, round(sleep_time, 2))
            current_time = 0
            
            print(f"\n--- Reaction {reaction_id} | Ratio: {ratio} | Dilution: {dilution} ---")

            # --- Folder Creation ---
            # We pass the formatted ID (01, 02...) and the raw parameters
            folder_path = create_folder(
                daily_directory, 
                f"{reaction_id:02d}", 
                ratio, 
                dilution, 
                frequency
            )
            
            # Create subfolders inside the new folder_path
            folder_path_raw = os.path.join(folder_path, "raw_data")
            folder_path_corrected = os.path.join(folder_path, "corrected_data")
            folder_path_duration = os.path.join(folder_path, "duration")

            for path in [folder_path_raw, folder_path_corrected, folder_path_duration]:
                if not os.path.exists(path):
                    os.makedirs(path)

            # Setup Data Structures (Dictionaries)
            intensities_nir_dict = {}
            intensities_vis_dict = {}
            nir_measurement_duration_dict = {}
            vis_measurement_duration_dict = {}
            
            # Define File Paths
            path_nir = os.path.join(folder_path_raw, "Emission_nir.csv")
            path_vis = os.path.join(folder_path_raw, "Emission_vis.csv")
            path_nir_corrected = os.path.join(folder_path_corrected, "Emission_nir_corrected.csv")
            path_vis_corrected = os.path.join(folder_path_corrected, "Emission_vis_corrected.csv")
            path_dur_nir = os.path.join(folder_path_duration, "nir_measurement_duration.csv")
            path_dur_vis = os.path.join(folder_path_duration, "vis_measurement_duration.csv")

            # Get Wavelengths
            wavelength_nir = nir.get_wavelengths()
            wavelength_vis = vis.get_wavelengths()

            # --- Reference Acquisition (Dark & Air) ---
            vis.get_formatted_spectrum() # Flush buffer
            nir.get_formatted_spectrum()
            time.sleep(1)
            dark_vis = np.array(vis.get_formatted_spectrum())
            dark_nir = np.array(nir.get_formatted_spectrum())

            # Stop threshold calc
            mean_dark_nir = np.mean(dark_nir)
            lower_bound = mean_dark_nir * (1 - intensity_threshold_percentage)
            upper_bound = mean_dark_nir * (1 + intensity_threshold_percentage)

            # Air Reference
            mux.set_trigger_out(True) 
            air_vis = np.array(vis.get_formatted_spectrum())
            air_nir = np.array(nir.get_formatted_spectrum())
            mux.set_trigger_out(False) 
                    
            # --- Reference Acquisition (Toluene) ---
            # NOTE: We always do the Reference Toluene measurement for calibration,
            # even if the reaction itself doesn't use Toluene. This ensures consistency.
            flow.switch_v1_tol_or_pre("tol")
            ob.set_pressure(1, 1000)
            time.sleep(5)
            flow.Tol_cuvette_volume(2000) # This fills the cuvette
            ob.set_pressure(1, 0)

            mux.set_trigger_out(True)
            toluene_vis = np.array(vis.get_formatted_spectrum())
            toluene_nir = np.array(nir.get_formatted_spectrum())
            mux.set_trigger_out(False)
            
            # Empty the cuvette after reference
            flow.cuvette_waste_or_tube(12, 15) 

            # Save Calibration Data
            df_vis = pd.DataFrame({"Wavelength": wavelength_vis, "Dark": dark_vis, "Air": air_vis, "Toluene": toluene_vis})
            df_vis.to_csv(os.path.join(folder_path_raw, "Vis_calibration.csv"), index=False)
            df_nir = pd.DataFrame({"Wavelength": wavelength_nir, "Dark": dark_nir, "Air": air_nir, "Toluene": toluene_nir})
            df_nir.to_csv(os.path.join(folder_path_raw, "Nir_calibration.csv"), index=False)

            # Calculate Corrected Calibration
            air_nir_corr = nir.nonlinearity_correct_spectrum2(dark_nir, air_nir)
            toluene_nir_corr = nir.nonlinearity_correct_spectrum2(dark_nir, toluene_nir)
            air_vis_corr = vis.nonlinearity_correct_spectrum2(dark_vis, air_vis)
            toluene_vis_corr = vis.nonlinearity_correct_spectrum2(dark_vis, toluene_vis)

            # Save Corrected Calibration
            df_vis_corr = pd.DataFrame({"Wavelength": wavelength_vis, "Dark": dark_vis, "Air_Corrected": air_vis_corr, "Toluene_Corrected": toluene_vis_corr})
            df_vis_corr.to_csv(os.path.join(folder_path_corrected, "Vis_calibration_corrected.csv"), index=False)
            df_nir_corr = pd.DataFrame({"Wavelength": wavelength_nir, "Dark": dark_nir, "Air_Corrected": air_nir_corr, "Toluene_Corrected": toluene_nir_corr})
            df_nir_corr.to_csv(os.path.join(folder_path_corrected, "Nir_calibration_corrected.csv"), index=False) 

            # --- Start Reaction Injection ---
            measurement_index = 0 
            print(" -> Injecting Precursors...")

            # 1. Toluene Injection
            if toluene_volume != 0:
                print(f"    Injecting Toluene ({toluene_volume:.1f} uL)...")
                flow.switch_v1_tol_or_pre("tol")
                ob.set_pressure(1, 1000)
                time.sleep(10)
                flow.Tol_cuvette_volume(toluene_volume)
                pass

            # 2. Precursor Injection
            flow.switch_v1_tol_or_pre("pre")
            ob.set_pressure(1, 1000)
            time.sleep(10) 

            flow.Ag_cuvette_volume(Ag_volume)
            time.sleep(5) 
            flow.S_cuvette_volume(S_volume)
            
            start_time = time.perf_counter()
            ob.set_pressure(1, 0) # Vent
            # --- Measurement Loop ---
            while current_time <= duration:
                # Trigger ON
                mux.set_trigger_out(True)
                time1 = time.perf_counter()
                
                # Acquire
                intensity_nir = np.array(nir.get_formatted_spectrum())
                time2 = time.perf_counter()
                intensity_vis = np.array(vis.get_formatted_spectrum())
                time3 = time.perf_counter()
                
                # Trigger OFF
                mux.set_trigger_out(False) 
                
                measurement_index += 1

                # Timing & Storage
                current_timestamp = round(time1 - start_time, 1)
                nir_dur = time2 - time1
                vis_dur = time3 - time2
                
                intensities_nir_dict[current_timestamp] = intensity_nir
                intensities_vis_dict[current_timestamp] = intensity_vis
                nir_measurement_duration_dict[current_timestamp] = nir_dur
                vis_measurement_duration_dict[current_timestamp] = vis_dur
                
                print(f"Timestamp: {current_timestamp}s | NIR: {nir_dur:.3f}s | VIS: {vis_dur:.3f}s", end='\r')

                # Stop Condition Logic
                mean_intensity_nir = np.mean(intensity_nir)
                last_mean_nir.append(mean_intensity_nir)
                
                if len(last_mean_nir) == time_threshold and current_timestamp > minimum_stop_time:
                    all_within_threshold = all(lower_bound <= val <= upper_bound for val in last_mean_nir)
                    if all_within_threshold:
                        print("\n*** Auto-Stop: NIR intensity baseline reached. ***\n")
                        break 

                if current_timestamp > duration:
                    break
                
                time.sleep(sleep_time)
            pass

        except FlowControlError as e:
            print(f"\n🔥 FLOW ERROR: {e}")
            try:
                ob.set_pressure(1, 0)
                ob.set_pressure(2, 0)
                mux.set_trigger_out(False) 
                flow.muxd.switch_valve(11, direction='short') 
                time.sleep(1) 
            except Exception as e:
                print(f"⚠️ Warning during safety shutdown: {e}")
            print("❌ Critical Hardware Failure. Stopping.")
            break
        
        except Exception as e:
            print(f"error {e} during reaction {reaction_id}")
            break

        # --- STANDARD CLEANUP (Success) ---
        print(" -> Cleaning up...")
        try:
            mux.set_trigger_out(False)
            flow.cuvette_waste_or_tube(tube_number, 15) # Product to tube
            
            # Flush Lines
            ob.set_pressure(1, 1000)
            flow.S_cuvette_volume(100)
            flow.Ag_cuvette_volume(100)
            
            # Wash Cycle
            flow.switch_v1_tol_or_pre("tol")
            ob.set_pressure(1, 1000)
            
            for _ in range(2):
                flow.toluene_cuvette(toluene_cleaning)
                flow.cuvette_waste_or_tube(12, 15) # Waste
            
            ob.set_pressure(1, 0)
            
        except Exception as e:
            print(f"⚠️ Cleanup Warning: {e}")

        # --- DATA SAVING (Identical to your code) ---
        try:
            reaction_parameters[reaction_id]['num_measurements'] = measurement_index
            
            # Update the CSV file
            df_params_update = pd.DataFrame.from_dict(reaction_parameters, orient="index")
            df_params_update.index.name = "reaction_id"
            df_params_update.to_csv(os.path.join(daily_directory, path_reaction_parameters))

            # Save Dicts to CSV
            pd.DataFrame.from_dict(intensities_nir_dict, orient='index', columns=wavelength_nir).to_csv(path_nir)
            pd.DataFrame.from_dict(intensities_vis_dict, orient='index', columns=wavelength_vis).to_csv(path_vis)
            
            # Correction & Save
            # (Loop for correction - same as yours)
            for k in intensities_nir_dict:
                intensities_nir_dict[k] = nir.nonlinearity_correct_spectrum2(dark_nir, intensities_nir_dict[k])
            for k in intensities_vis_dict:
                intensities_vis_dict[k] = vis.nonlinearity_correct_spectrum2(dark_vis, intensities_vis_dict[k])
                
            pd.DataFrame.from_dict(intensities_nir_dict, orient='index', columns=wavelength_nir).round(1).to_csv(path_nir_corrected)
            pd.DataFrame.from_dict(intensities_vis_dict, orient='index', columns=wavelength_vis).round(1).to_csv(path_vis_corrected)

            print(f"💾 Data saved for Reaction {reaction_id}.")
            
            # Backup to Network
            try:
                dest_dir = r"\\Gfs01\g11\FluoSpec\Alle\Alex_Relling\Austausch\kinetic_setup_data"
                dest_path = os.path.join(dest_dir, date_str)
                # Using copytree with dirs_exist_ok=True to merge/update
                shutil.copytree(daily_directory, dest_path, dirs_exist_ok=True)
            except Exception as e:
                print(f"❌ Network Backup Failed: {e}")

        except Exception as e:
            print(f"❌ Local Save Failed: {e}")

except KeyboardInterrupt:
    print("\n🛑 Manual Stop.")

finally:
    # SHUTDOWN SEQUENCE
    print("🔻 System Shutdown.")
    try:
        mux.set_trigger_out(False)
        flow.stop()
        ob.set_pressure(1, 0)
        stirrer.stop_stirring()
        
        # Final Wash
        flow.switch_v1_tol_or_pre("tol")
        ob.set_pressure(1, 1000)
        flow.cuvette_waste_or_tube(12, 15)
        for i in range(3):
            flow.toluene_cuvette(toluene_cleaning)
            flow.cuvette_waste_or_tube(12, 15)
        flow.stop()
    except Exception as e:
        print(f"error {e}")
        flow.stop()
        

🚀 Starting Main Experiment Loop...
⏹️ Stirring Stopped
✅ Stirring set to 400 RPM
✅ Setup Complete. Parameters and Calibration saved to C:\data\arelling\2025-12-01

--- Reaction 1 | Ratio: 10.0 | Dilution: 1 ---
 -> Injecting Precursors...
 -> Cleaning up... | NIR: 0.203s | VIS: 0.190s
💾 Data saved for Reaction 1.

--- Reaction 2 | Ratio: 9.0 | Dilution: 1 ---
 -> Injecting Precursors...


In [ ]:
ctypes.windll.kernel32.SetThreadExecutionState(0x80000000)